In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove nitin commented as its not used
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

# from keras.optimizers import Adam # nitin commented as Adam has been shifted to optimizer_v1 module.
from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
#from keras.backend.tensorflow_backend import set_session, clear_session, get_session # nitin commented as tensorflow_backend not used anymore
from keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# nitin added as without it the read_pickel below was giving the error as in notebook 5 when its writing
#pickel do a lazy execution and write the function as well
def mean(a):
    return sum(a) / len(a)

In [3]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")


ner_word2vec = pd.read_pickle("data/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle("data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")
ner_concat = pd.read_pickle("data/"+type_of_ner+"_ner_combined_limited_dict.pkl")

train_ids = pd.read_pickle("data/"+type_of_ner+"_train_ids.pkl")
dev_ids = pd.read_pickle("data/"+type_of_ner+"_dev_ids.pkl")
test_ids = pd.read_pickle("data/"+type_of_ner+"_test_ids.pkl")

In [4]:
x_train_lstm.shape,x_dev_lstm.shape,x_test_lstm.shape ,y_train.shape,y_dev.shape,y_test.shape

((14665, 24, 104),
 (2090, 24, 104),
 (4171, 24, 104),
 (14665, 4),
 (2090, 4),
 (4171, 4))

In [5]:
def make_prediction_cnn(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_cnn(predictions, probs, ground_truth, 
                          
                          embed_name, problem_type, iteration, hidden_unit_size,
                          
                          sequence_name, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

    result_path = "results/cnn/"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+"-cnn-.p"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)
    
def print_scores_cnn(predictions, probs, ground_truth, model_name, problem_type, iteration, hidden_unit_size):
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    print ("AUC: ", auc, "AUPRC: ", auprc, "F1: ", F1)
    
def get_subvector_data(size, embed_name, data):
    if embed_name == "concat":
        vector_size = 200
    else:
        vector_size = 100

    x_data = {}
    for k, v in data.items():
        number_of_additional_vector = len(v) - size
        vector = []
        for i in v:
            vector.append(i)
        if number_of_additional_vector < 0: 
            number_of_additional_vector = np.abs(number_of_additional_vector)

            temp = vector[:size]
            for i in range(0, number_of_additional_vector):
                temp.append(np.zeros(vector_size))
            x_data[k] = np.asarray(temp)
        else:
            x_data[k] = np.asarray(vector[:size])

    return x_data


def proposedmodel(layer_name, number_of_unit, embedding_name, ner_limit, num_filter):
    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_img = Input(shape=(ner_limit, input_dimension), name = "cnn_input")

    convs = []
    filter_sizes = [2,3,4]


    #logits_regularizer = tf.compat.v1.estimator.layers.l2_regularizer(scale=0.01)
    # removed tf.compat.v1.estimator.layers.xavier_initializer() with glorot_normal
    
    text_conv1d = Conv1D(filters=num_filter, kernel_size=3, 
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu', 
                         kernel_initializer=tf.keras.initializers.glorot_normal() )(input_img)
    
    text_conv1d = Conv1D(filters=num_filter*2, kernel_size=3, 
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=tf.keras.initializers.glorot_normal())(text_conv1d)   
    
    text_conv1d = Conv1D(filters=num_filter*3, kernel_size=3, 
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=tf.keras.initializers.glorot_normal())(text_conv1d)   

    
    #concat_conv = keras.layers.Concatenate()([text_conv1d, text_conv1d_2, text_conv1d_3])
    text_embeddings = GlobalMaxPooling1D()(text_conv1d)
    #text_embeddings = Dense(128, activation="relu")(text_embeddings)
    
    if layer_name == "GRU":
        x = GRU(number_of_unit)(sequence_input)
    elif layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)

    concatenated = keras.layers.Concatenate(axis =1)([x, text_embeddings])# nitin uncommented
    #concatenated = merge([x, text_embeddings], mode='concat', concat_axis=1)# nitin commented

    concatenated = Dense(512, activation='relu')(concatenated)
    concatenated = Dropout(0.2)(concatenated)
    #concatenated = Dense(256, activation='relu')(concatenated)
    #concatenated = Dense(512, activation='relu')(concatenated)
    
    #concatenated = Dense(512, activation='relu')(concatenated)
    
    #logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01) nitin commented as its deprecated
    logits_regularizer = tf.keras.regularizers.L2(0.01)
    
    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.glorot_normal(), 
                  kernel_regularizer=logits_regularizer)(concatenated)
    
    
    #opt = Adam(lr=1e-4, decay = 0.01)
    
    opt = Adam(lr=1e-3, decay = 0.01)
    
    #opt = Adam(lr=0.001)

    model = Model(inputs=[sequence_input, input_img], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])
    
    return model


dict_f = {"a": [1,2,3], "b": [1,2,3], "c": [1,2,3], "d": [1,2,3]}

#data = list(dict.items())
data = collections.OrderedDict(sorted(dict_f.items()))
print(sorted(dict_f.items()))
an_array = np.asarray(data.values())

print(data)
print('an_array',an_array)
print(type(data))



for a in data.items():
    print(a[1])

t1[0]

i =0
t = []
for kk,v in x_train_dict_sorted.items():
    i+=1
    #print(kk, v.shape,i+1)
    t.append(v)
a  = np.array(t)
    

a.shape

print(a)

In [6]:
embedding_types = ['word2vec', 'fasttext']#, 'concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]

target_problems = ['mort_hosp', 'mort_icu']#, 'los_3', 'los_7']

num_epoch = 100
model_patience = 5
monitor_criteria = 'val_loss'
#monitor_criteria = 'val_acc'
batch_size = 64

filter_number = 32
ner_representation_limit = 64
activation_func = "relu"

sequence_model = "GRU"
sequence_hidden_unit = 256

maxiter = 11
for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
    print ("Embedding: ", embed_name)
    print("=============================")
    
    temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
    tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
    temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

    x_train_dict = {}
    x_dev_dict = {}
    x_test_dict = {}

    x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
    x_dev_dict = get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
    x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

    x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
    x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
    x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))
    
#     x_train_dict_sorted = sorted(x_train_dict.items())
#     x_dev_dict_sorted = sorted(x_dev_dict.items())
#     x_test_dict_sorted = sorted(x_test_dict.items())
    list_train_ner =[]
    list_test_ner =[]
    list_dev_ner =[]
    for k,v in x_train_dict_sorted.items():
            list_train_ner.append(v)
    for k,v in x_test_dict_sorted.items():
            list_test_ner.append(v)
    for k,v in x_dev_dict_sorted.items():
            list_dev_ner.append(v)
    
        
    
    x_train_ner = np.array(list_train_ner)
    x_dev_ner = np.array(list_dev_ner)
    x_test_ner = np.array(list_test_ner)
    
    print(x_train_ner.shape)
    

    #print('shape of x_train_ner{0} x_dev_ner{1} x_test_ner {2}'.format(x_train_ner.shape,x_dev_ner.shape,x_test_ner.shape))
    
        
    for iteration in range(1,maxiter):
        print ("Iteration number: ", iteration)
    
        for each_problem in target_problems:
            print ("Problem type: ", each_problem)
            print ("__________________")
            
            
            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
            
            best_model_name = str(ner_representation_limit)+"-basiccnn1d-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
            
            checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
                save_best_only=True, mode='min')
            
            reduce_lr = ReduceLROnPlateau(monitor=monitor_criteria, factor=0.2,
                              patience=2, min_lr=0.00001, epsilon=1e-4, mode='min')
            

            callbacks = [early_stopping_monitor, checkpoint, reduce_lr]
            
            #model = textCNN(sequence_model, sequence_hidden_unit, embed_name, ner_representation_limit)
            model = proposedmodel(sequence_model, sequence_hidden_unit, 
                               embed_name, ner_representation_limit,filter_number)
            print('x_train_lstm {0} x_dev_lstm {1}'.format(x_train_lstm.shape,x_dev_lstm.shape))
            print('x_train_ner {0} x_dev_ner {1}'.format(x_train_ner.shape,x_dev_ner.shape))
            print(y_train[each_problem].shape)
            aa = [x_train_lstm, x_train_ner]
            print(x_train_lstm.shape, x_train_ner.shape, len(aa))
            model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                      validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, batch_size=batch_size)
            
            
            probs, predictions = make_prediction_cnn(model, [x_test_lstm, x_test_ner])
            print_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, sequence_hidden_unit)
            
            model.load_weights(best_model_name)
                      
            probs, predictions = make_prediction_cnn(model, [x_test_lstm, x_test_ner])
            save_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
                            sequence_hidden_unit, sequence_model, type_of_ner)
            del model
            clear_session()
            gc.collect()
            
    

Embedding:  word2vec
(14665, 64, 100)
Iteration number:  1
Problem type:  mort_hosp
__________________
x_train_lstm (14665, 24, 104) x_dev_lstm (2090, 24, 104)
x_train_ner (14665, 64, 100) x_dev_ner (2090, 64, 100)
(14665,)
(14665, 24, 104) (14665, 64, 100) 2
Epoch 1/100
229/230 [============================>.] - ETA: 0s - loss: 0.2944 - acc: 0.8998
Epoch 00001: val_loss improved from inf to 0.24731, saving model to 64-basiccnn1d-word2vec-mort_hosp-best_model.hdf5
230/230 [==============================] - 20s 73ms/step - loss: 0.2945 - acc: 0.8998 - val_loss: 0.2473 - val_acc: 0.9158 - lr: 0.0010
Epoch 2/100
229/230 [============================>.] - ETA: 0s - loss: 0.2381 - acc: 0.9157
Epoch 00002: val_loss improved from 0.24731 to 0.24633, saving model to 64-basiccnn1d-word2vec-mort_hosp-best_model.hdf5
230/230 [==============================] - 16s 69ms/step - loss: 0.2381 - acc: 0.9156 - val_loss: 0.2463 - val_acc: 0.9139 - lr: 0.0010
Epoch 3/100
229/230 [=========================

229/230 [============================>.] - ETA: 0s - loss: 0.1872 - acc: 0.9333
Epoch 00006: val_loss did not improve from 0.24186
230/230 [==============================] - 13s 54ms/step - loss: 0.1873 - acc: 0.9333 - val_loss: 0.2428 - val_acc: 0.9105 - lr: 2.0000e-04
Epoch 7/100
229/230 [============================>.] - ETA: 0s - loss: 0.1849 - acc: 0.9334
Epoch 00007: val_loss did not improve from 0.24186
230/230 [==============================] - 13s 55ms/step - loss: 0.1848 - acc: 0.9334 - val_loss: 0.2430 - val_acc: 0.9115 - lr: 2.0000e-04
Epoch 8/100
229/230 [============================>.] - ETA: 0s - loss: 0.1838 - acc: 0.9335
Epoch 00008: val_loss did not improve from 0.24186
230/230 [==============================] - 13s 57ms/step - loss: 0.1838 - acc: 0.9335 - val_loss: 0.2429 - val_acc: 0.9110 - lr: 4.0000e-05
AUC:  0.8799214987586151 AUPRC:  0.5841802257497581 F1:  0.471281296023564
0.8811399058974912 0.5904104572633758 0.9144090146247902 0.45827010622154774
Problem typ

Epoch 4/100
229/230 [============================>.] - ETA: 0s - loss: 0.1501 - acc: 0.9484- ETA: 1s - 
Epoch 00004: val_loss did not improve from 0.17055
230/230 [==============================] - 15s 66ms/step - loss: 0.1500 - acc: 0.9484 - val_loss: 0.1714 - val_acc: 0.9426 - lr: 0.0010
Epoch 5/100
229/230 [============================>.] - ETA: 0s - loss: 0.1409 - acc: 0.9520
Epoch 00005: val_loss did not improve from 0.17055
230/230 [==============================] - 15s 67ms/step - loss: 0.1409 - acc: 0.9519 - val_loss: 0.1708 - val_acc: 0.9426 - lr: 0.0010
Epoch 6/100
229/230 [============================>.] - ETA: 0s - loss: 0.1310 - acc: 0.9548
Epoch 00006: val_loss did not improve from 0.17055
230/230 [==============================] - 16s 69ms/step - loss: 0.1309 - acc: 0.9548 - val_loss: 0.1707 - val_acc: 0.9421 - lr: 2.0000e-04
Epoch 7/100
229/230 [============================>.] - ETA: 0s - loss: 0.1301 - acc: 0.9536
Epoch 00007: val_loss did not improve from 0.17055
230/

229/230 [============================>.] - ETA: 0s - loss: 0.2927 - acc: 0.9011
Epoch 00001: val_loss improved from inf to 0.25107, saving model to 64-basiccnn1d-word2vec-mort_hosp-best_model.hdf5
230/230 [==============================] - 19s 64ms/step - loss: 0.2926 - acc: 0.9011 - val_loss: 0.2511 - val_acc: 0.9158 - lr: 0.0010
Epoch 2/100
229/230 [============================>.] - ETA: 0s - loss: 0.2388 - acc: 0.9157
Epoch 00002: val_loss improved from 0.25107 to 0.24160, saving model to 64-basiccnn1d-word2vec-mort_hosp-best_model.hdf5
230/230 [==============================] - 15s 67ms/step - loss: 0.2387 - acc: 0.9157 - val_loss: 0.2416 - val_acc: 0.9129 - lr: 0.0010
Epoch 3/100
229/230 [============================>.] - ETA: 0s - loss: 0.2250 - acc: 0.9198- ET
Epoch 00003: val_loss did not improve from 0.24160
230/230 [==============================] - 15s 67ms/step - loss: 0.2249 - acc: 0.9198 - val_loss: 0.2429 - val_acc: 0.9124 - lr: 0.0010
Epoch 4/100
229/230 [==============

229/230 [============================>.] - ETA: 0s - loss: 0.1843 - acc: 0.9327
Epoch 00006: val_loss did not improve from 0.23887
230/230 [==============================] - 16s 68ms/step - loss: 0.1842 - acc: 0.9327 - val_loss: 0.2411 - val_acc: 0.9148 - lr: 2.0000e-04
Epoch 7/100
229/230 [============================>.] - ETA: 0s - loss: 0.1804 - acc: 0.9344- ETA: 1s - los
Epoch 00007: val_loss did not improve from 0.23887
230/230 [==============================] - 15s 65ms/step - loss: 0.1805 - acc: 0.9344 - val_loss: 0.2418 - val_acc: 0.9124 - lr: 2.0000e-04
Epoch 8/100
229/230 [============================>.] - ETA: 0s - loss: 0.1782 - acc: 0.9356
Epoch 00008: val_loss did not improve from 0.23887
230/230 [==============================] - 16s 68ms/step - loss: 0.1782 - acc: 0.9356 - val_loss: 0.2418 - val_acc: 0.9120 - lr: 4.0000e-05
AUC:  0.8837081054711664 AUPRC:  0.580418834325865 F1:  0.47214076246334313
0.8845924522387936 0.5853172232284884 0.917286022536562 0.47004608294930

...

Epoch 9/100
229/230 [============================>.] - ETA: 0s - loss: 0.1782 - acc: 0.9336
Epoch 00009: val_loss did not improve from 0.24167
230/230 [==============================] - 28s 120ms/step - loss: 0.1782 - acc: 0.9336 - val_loss: 0.2465 - val_acc: 0.9100 - lr: 4.0000e-05
AUC:  0.8753318509772592 AUPRC:  0.5795877147580027 F1:  0.4826589595375722
0.8768236337991325 0.5863836218072976 0.917286022536562 0.4888888888888888
Problem type:  mort_icu
__________________
x_train_lstm (14665, 24, 104) x_dev_lstm (2090, 24, 104)
x_train_ner (14665, 64, 100) x_dev_ner (2090, 64, 100)
(14665,)
(14665, 24, 104) (14665, 64, 100) 2
Epoch 1/100
229/230 [============================>.] - ETA: 0s - loss: 0.2244 - acc: 0.9320
Epoch 00001: val_loss improved from inf to 0.18114, saving model to 64-basiccnn1d-fasttext-mort_icu-best_model.hdf5
230/230 [==============================] - 30s 114ms/step - loss: 0.2246 - acc: 0.9320 - val_loss: 0.1811 - val_acc: 0.9402 - lr: 0.0010
Epoch 2/100
229/230 

Epoch 5/100
229/230 [============================>.] - ETA: 0s - loss: 0.1397 - acc: 0.9517
Epoch 00005: val_loss did not improve from 0.17480
230/230 [==============================] - 27s 116ms/step - loss: 0.1396 - acc: 0.9517 - val_loss: 0.1763 - val_acc: 0.9402 - lr: 0.0010
Epoch 6/100
229/230 [============================>.] - ETA: 0s - loss: 0.1316 - acc: 0.9565
Epoch 00006: val_loss did not improve from 0.17480
230/230 [==============================] - 25s 108ms/step - loss: 0.1316 - acc: 0.9565 - val_loss: 0.1769 - val_acc: 0.9416 - lr: 0.0010
Epoch 7/100
229/230 [============================>.] - ETA: 0s - loss: 0.1220 - acc: 0.9567
Epoch 00007: val_loss did not improve from 0.17480
230/230 [==============================] - 25s 110ms/step - loss: 0.1220 - acc: 0.9567 - val_loss: 0.1776 - val_acc: 0.9416 - lr: 2.0000e-04
Epoch 8/100
229/230 [============================>.] - ETA: 0s - loss: 0.1201 - acc: 0.9589
Epoch 00008: val_loss did not improve from 0.17480
230/230 [====

229/230 [============================>.] - ETA: 0s - loss: 0.2208 - acc: 0.9213
Epoch 00003: val_loss improved from 0.24498 to 0.23958, saving model to 64-basiccnn1d-fasttext-mort_hosp-best_model.hdf5
230/230 [==============================] - 24s 105ms/step - loss: 0.2209 - acc: 0.9213 - val_loss: 0.2396 - val_acc: 0.9134 - lr: 0.0010
Epoch 4/100
229/230 [============================>.] - ETA: 0s - loss: 0.2100 - acc: 0.9264
Epoch 00004: val_loss did not improve from 0.23958
230/230 [==============================] - 24s 104ms/step - loss: 0.2099 - acc: 0.9264 - val_loss: 0.2461 - val_acc: 0.9153 - lr: 0.0010
Epoch 5/100
229/230 [============================>.] - ETA: 0s - loss: 0.1977 - acc: 0.9285
Epoch 00005: val_loss did not improve from 0.23958
230/230 [==============================] - 23s 101ms/step - loss: 0.1980 - acc: 0.9284 - val_loss: 0.2411 - val_acc: 0.9144 - lr: 0.0010
Epoch 6/100
229/230 [============================>.] - ETA: 0s - loss: 0.1860 - acc: 0.9327
Epoch 0000

229/230 [============================>.] - ETA: 0s - loss: 0.1753 - acc: 0.9387- ETA: 5s -  - ETA: 2
Epoch 00009: val_loss did not improve from 0.24458
230/230 [==============================] - 22s 95ms/step - loss: 0.1754 - acc: 0.9386 - val_loss: 0.2498 - val_acc: 0.9081 - lr: 4.0000e-05
AUC:  0.8772749393447703 AUPRC:  0.5818694160569384 F1:  0.4787077826725404
0.8775288723606579 0.5842044493217542 0.9148885159434188 0.4563552833078101
Problem type:  mort_icu
__________________
x_train_lstm (14665, 24, 104) x_dev_lstm (2090, 24, 104)
x_train_ner (14665, 64, 100) x_dev_ner (2090, 64, 100)
(14665,)
(14665, 24, 104) (14665, 64, 100) 2
Epoch 1/100
229/230 [============================>.] - ETA: 0s - loss: 0.2166 - acc: 0.9351
Epoch 00001: val_loss improved from inf to 0.17807, saving model to 64-basiccnn1d-fasttext-mort_icu-best_model.hdf5
230/230 [==============================] - 28s 111ms/step - loss: 0.2165 - acc: 0.9352 - val_loss: 0.1781 - val_acc: 0.9421 - lr: 0.0010
Epoch 2/100

229/230 [============================>.] - ETA: 0s - loss: 0.1309 - acc: 0.9556
Epoch 00006: val_loss did not improve from 0.17289
230/230 [==============================] - 24s 106ms/step - loss: 0.1309 - acc: 0.9555 - val_loss: 0.1747 - val_acc: 0.9383 - lr: 2.0000e-04
Epoch 7/100
229/230 [============================>.] - ETA: 0s - loss: 0.1288 - acc: 0.9559
Epoch 00007: val_loss did not improve from 0.17289
230/230 [==============================] - 25s 110ms/step - loss: 0.1289 - acc: 0.9558 - val_loss: 0.1753 - val_acc: 0.9383 - lr: 2.0000e-04
Epoch 8/100
229/230 [============================>.] - ETA: 0s - loss: 0.1265 - acc: 0.9569
Epoch 00008: val_loss did not improve from 0.17289
230/230 [==============================] - 25s 110ms/step - loss: 0.1267 - acc: 0.9568 - val_loss: 0.1753 - val_acc: 0.9373 - lr: 4.0000e-05
AUC:  0.8855168726136466 AUPRC:  0.5279251938339963 F1:  0.46551724137931033
0.8817713946746205 0.520536113617849 0.9405418364900503 0.4464285714285714
Iteratio

embedding_types = ['word2vec', 'fasttext', 'concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]

target_problems = [ 'mort_icu', 'los_3', 'los_7']#['mort_hosp']#,

num_epoch = 100
model_patience = 5
monitor_criteria = 'val_loss'
#monitor_criteria = 'val_acc'
batch_size = 64

filter_number = 32
ner_representation_limit = 64
activation_func = "relu"

sequence_model = "GRU"
sequence_hidden_unit = 256

maxiter = 11
for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
    print ("Embedding: ", embed_name)
    print("=============================")
    
    temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
    tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
    temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

    x_train_dict = {}
    x_dev_dict = {}
    x_test_dict = {}

    x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
    x_dev_dict = get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
    x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

    x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
    x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
    x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))
    
#     x_train_dict_sorted = sorted(x_train_dict.items())
#     x_dev_dict_sorted = sorted(x_dev_dict.items())
#     x_test_dict_sorted = sorted(x_test_dict.items())
    list_train_ner =[]
    list_test_ner =[]
    list_dev_ner =[]
    for k,v in x_train_dict_sorted.items():
            list_train_ner.append(v)
    for k,v in x_test_dict_sorted.items():
            list_test_ner.append(v)
    for k,v in x_dev_dict_sorted.items():
            list_dev_ner.append(v)
    
        
    
    x_train_ner = np.array(list_train_ner)
    x_dev_ner = np.array(list_dev_ner)
    x_test_ner = np.array(list_test_ner)
    
    print(x_train_ner.shape)
    

    #print('shape of x_train_ner{0} x_dev_ner{1} x_test_ner {2}'.format(x_train_ner.shape,x_dev_ner.shape,x_test_ner.shape))
    
        
    for iteration in range(1,maxiter):
        print ("Iteration number: ", iteration)
    
        for each_problem in target_problems:
            print ("Problem type: ", each_problem)
            print ("__________________")
            
            
            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
            
            best_model_name = str(ner_representation_limit)+"-basiccnn1d-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
            
            checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
                save_best_only=True, mode='min')
            
            reduce_lr = ReduceLROnPlateau(monitor=monitor_criteria, factor=0.2,
                              patience=2, min_lr=0.00001, epsilon=1e-4, mode='min')
            

            callbacks = [early_stopping_monitor, checkpoint, reduce_lr]
            
            #model = textCNN(sequence_model, sequence_hidden_unit, embed_name, ner_representation_limit)
            model = proposedmodel(sequence_model, sequence_hidden_unit, 
                               embed_name, ner_representation_limit,filter_number)
            print('x_train_lstm {0} x_dev_lstm {1}'.format(x_train_lstm.shape,x_dev_lstm.shape))
            print('x_train_ner {0} x_dev_ner {1}'.format(x_train_ner.shape,x_dev_ner.shape))
            print(y_train[each_problem].shape)
            aa = [x_train_lstm, x_train_ner]
            print(x_train_lstm.shape, x_train_ner.shape, len(aa))
            model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                      validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, batch_size=batch_size)
            
            
            probs, predictions = make_prediction_cnn(model, [x_test_lstm, x_test_ner])
            print_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, sequence_hidden_unit)
            
            model.load_weights(best_model_name)
                      
            probs, predictions = make_prediction_cnn(model, [x_test_lstm, x_test_ner])
            save_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
                            sequence_hidden_unit, sequence_model, type_of_ner)
            del model
            clear_session()
            gc.collect()
            